# Example of the Conditional Logit Model on ModeCanada Dataset
This tutorial is modified from the [Random utility model and the multinomial logit model](https://cran.r-project.org/web/packages/mlogit/vignettes/c3.rum.html) in th documentation of `mlogit` package in R. Please note that the dataset involved in this example is fairly small (2,779 choice records), so we don't expect the performance to be faster than the R implementation. We provide this tutorial mainly to check the correctness of our prediction. The fully potential of PyTorch is better exploited on much larger dataset.

In [1]:
import argparse

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

from torch_choice.data import ChoiceDataset, utils
from torch_choice.model import ConditionalLogitModel

from torch_choice.utils.run_helper import run

In [2]:
if torch.cuda.is_available():
    print(f'CUDA device used: {torch.cuda.get_device_name()}')

CUDA device used: NVIDIA GeForce RTX 3090


In [3]:
args = argparse.Namespace(data_path='./',
                          batch_size=-1,  # full-batch.
                          shuffle=False,
                          num_epochs=5000,
                          device='cuda' if torch.cuda.is_available() else 'cpu')

## Load Dataset

In [4]:
df = pd.read_csv('./public_datasets/ModeCanada.csv', index_col=0)
df = df.query('noalt == 4').reset_index(drop=True)
df.sort_values(by='case', inplace=True)
df.head()

,case,alt,choice,dist,cost,ivt,ovt,freq,income,urban,noalt
0,109,train,0,377,58.25,215,74,4,45,0,4
1,109,air,1,377,142.80,56,85,9,45,0,4
2,109,bus,0,377,27.52,301,63,8,45,0,4
3,109,car,0,377,71.63,262,0,0,45,0,4
4,110,train,0,377,58.25,215,74,4,70,0,4


In [5]:
df.shape

(11116, 11)

In [6]:
label = df[df['choice'] == 1].sort_values(by='case')['alt'].reset_index(drop=True)
print(f"{label=:}")

label=0       air
1       air
2       air
3       air
4       air
       ... 
2774    car
2775    car
2776    car
2777    car
2778    car
Name: alt, Length: 2779, dtype: object


In [7]:
item_names = ['air', 'bus', 'car', 'train']
num_items = 4
encoder = dict(zip(item_names, range(num_items)))
print(f"{encoder=:}")
label = label.map(lambda x: encoder[x])
label = torch.LongTensor(label)
print(f"{label=:}")

encoder={'air': 0, 'bus': 1, 'car': 2, 'train': 3}
label=tensor([0, 0, 0,  ..., 2, 2, 2])


In [8]:
price_cost_freq_ovt = utils.pivot3d(df, dim0='case', dim1='alt',
                                    values=['cost', 'freq', 'ovt'])
# session_income = torch.Tensor(df[['income']].values).view(-1, 1)
session_income = df.groupby('case')['income'].first()
session_income = torch.Tensor(session_income.values).view(-1, 1)
price_ivt = utils.pivot3d(df, dim0='case', dim1='alt', values='ivt')

In [9]:
dataset= ChoiceDataset(label=label,
                       price_cost_freq_ovt=price_cost_freq_ovt,
                       session_income=session_income,
                       price_ivt=price_ivt
                       ).to(args.device)
# data_loader = utils.create_data_loader(dataset, batch_size=-1, shuffle=True)

In [10]:
dataset

ChoiceDataset(label=[2779], user_index=[], session_index=[2779], item_availability=[], observable_prefix=[5], price_cost_freq_ovt=[2779, 4, 3], session_income=[2779, 1], price_ivt=[2779, 4, 1], device=cuda:0)

## Create the Model

In [11]:
model = ConditionalLogitModel(coef_variation_dict={'price_cost_freq_ovt': 'constant',
                                                   'session_income': 'item',
                                                   'price_ivt': 'item-full',
                                                   'intercept': 'item'},
                              num_param_dict={'price_cost_freq_ovt': 3,
                                              'session_income': 1,
                                              'price_ivt': 1,
                                              'intercept': 1},
                              num_items=4)

model = model.to(args.device)

In [12]:
run(model, dataset, num_epochs=10000)

==================== received model ====================
ConditionalLogitModel(
  (coef_dict): ModuleDict(
    (price_cost_freq_ovt): Coefficient(variation=constant, num_items=4, num_users=None, num_params=3, 3 trainable parameters in total).
    (session_income): Coefficient(variation=item, num_items=4, num_users=None, num_params=1, 3 trainable parameters in total).
    (price_ivt): Coefficient(variation=item-full, num_items=4, num_users=None, num_params=1, 4 trainable parameters in total).
    (intercept): Coefficient(variation=item, num_items=4, num_users=None, num_params=1, 3 trainable parameters in total).
  )
)
Conditional logistic discrete choice model, expects input features:

X[price_cost_freq_ovt] with 3 parameters, with constant level variation.
X[session_income] with 1 parameters, with item level variation.
X[price_ivt] with 1 parameters, with item-full level variation.
X[intercept] with 1 parameters, with item level variation.
==================== received dataset ========

## Train the Model
The model takes in a `ChoiceDataset` object in and return predicted potentials for each alternatives. The shape of output would be `len(dataset) X num_items`.

## Parameter Estimation

### R Output
```r
install.packages("mlogit")
library("mlogit")
data("ModeCanada", package = "mlogit")
MC <- dfidx(ModeCanada, subset = noalt == 4)
ml.MC1 <- mlogit(choice ~ cost + freq + ovt | income | ivt, MC, reflevel='air')

summary(ml.MC1)
```
```
Call:
mlogit(formula = choice ~ cost + freq + ovt | income | ivt, data = MC, 
    reflevel = "air", method = "nr")

Frequencies of alternatives:choice
      air     train       bus       car 
0.3738755 0.1666067 0.0035984 0.4559194 

nr method
9 iterations, 0h:0m:0s 
g'(-H)^-1g = 0.00014 
successive function values within tolerance limits 

Coefficients :
                    Estimate Std. Error  z-value  Pr(>|z|)    
(Intercept):train  3.2741952  0.6244152   5.2436 1.575e-07 ***
(Intercept):bus    0.6983381  1.2802466   0.5455 0.5854292    
(Intercept):car    1.8441129  0.7085089   2.6028 0.0092464 ** 
cost              -0.0333389  0.0070955  -4.6986 2.620e-06 ***
freq               0.0925297  0.0050976  18.1517 < 2.2e-16 ***
ovt               -0.0430036  0.0032247 -13.3356 < 2.2e-16 ***
income:train      -0.0381466  0.0040831  -9.3426 < 2.2e-16 ***
income:bus        -0.0890867  0.0183471  -4.8556 1.200e-06 ***
income:car        -0.0279930  0.0038726  -7.2286 4.881e-13 ***
ivt:air            0.0595097  0.0100727   5.9080 3.463e-09 ***
ivt:train         -0.0014504  0.0011875  -1.2214 0.2219430    
ivt:bus           -0.0067835  0.0044334  -1.5301 0.1259938    
ivt:car           -0.0064603  0.0018985  -3.4029 0.0006668 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Log-Likelihood: -1874.3
McFadden R^2:  0.35443 
Likelihood ratio test : chisq = 2058.1 (p.value = < 2.22e-16)
```